In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import backend as K
import os
import numpy as np
import uuid
import re

Using TensorFlow backend.


In [2]:
from trainlib.ModelCollection import ModelCollection
from trainlib.ModelCollectionConfigFileHandler import ModelCollectionConfigFileHandler

Welcome to JupyROOT 6.10/09


In [3]:
confhandler = ModelCollectionConfigFileHandler()

In [7]:
run_dir = "/data_CMS/cms/wind/180507_bkg_cat_ZXonly_manyepochs/number_layers_2.0_number_neurons_64.0/"

In [8]:
confhandler.load_configuration(os.path.join(run_dir, "settings.conf"))

attempting to load configuration file from /data_CMS/cms/wind/180507_bkg_cat_ZXonly_manyepochs/number_layers_2.0_number_neurons_64.0/settings.conf


In [9]:
mcolls = confhandler.GetModelCollection(weightpath = os.path.join(run_dir, "training/"))

got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 64.0}
building network with 28 inputs
FlexiblePCAPreprocessor set up for 28 inputs
got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 64.0}
building network with 10 inputs
FlexiblePCAPreprocessor set up for 10 inputs
got the following list of hyperparams: {u'number_layers': 2.0, u'number_neurons': 64.0}
building network with 15 inputs
FlexiblePCAPreprocessor set up for 15 inputs
found the following models belonging to this collection:
D_VBF_ggH_2j_ML
D_VBF_ggH_0j_ML
D_VBF_ggH_1j_ML
now attempting to load model from file /data_CMS/cms/wind/180507_bkg_cat_ZXonly_manyepochs/number_layers_2.0_number_neurons_64.0/training/D_VBF_ggH_ML/D_VBF_ggH_2j_ML/final.hdf5
read weights from /data_CMS/cms/wind/180507_bkg_cat_ZXonly_manyepochs/number_layers_2.0_number_neurons_64.0/training/D_VBF_ggH_ML/D_VBF_ggH_2j_ML/final.hdf5
now attempting to load column settings from file /data_CMS/cms/wi

2018-05-08 17:16:45.761504: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-05-08 17:16:45.761527: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-05-08 17:16:45.761533: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.


In [10]:
network_name = "D_VBF_ggH_2j_ML"

In [11]:
# find the desired neural network, as given by its name; and hold on to the network itself and its preprocessor

In [12]:
for mcoll in mcolls:
    if network_name in mcoll.model_dict.keys():
        found_mcoll = mcoll
        model = mcoll.model_dict[network_name]
        pre = mcoll.preprocessor_dict[network_name]
        break

In [13]:
pre.nonperiodic_columns

[u'JetPt[JetPt|2]',
 u'PFMET',
 u'D_VBF2j_ZHh_ME',
 u'D_VBF2j_WHh_ME',
 u'D_VBF2j_ggH_ME',
 u'JetPt[JetPt|0]',
 u'Z2Mass',
 u'JetEta[JetPt|1]',
 u'Z1Pt',
 u'ZZMassErr',
 u'JetPt[JetPt|1]',
 u'D_WHh_ggH_ME',
 u'D_ZHh_ggH_ME',
 u'Z1Mass',
 u'Z2Pt',
 u'JetEta[JetPt|2]',
 u'ZZPt',
 u'D_WHh_ZHh_ME',
 u'ZZEta',
 u'JetEta[JetPt|0]']

In [14]:
pre.periodic_columns

[u'JetPhi[JetPt|2]', u'JetPhi[JetPt|0]', u'ZZPhi', u'JetPhi[JetPt|1]']

In [53]:
# the actual names of the dataframe columns
column_names = list(set(map(lambda x: re.sub('\[.*\]', '', x), pre.periodic_columns + pre.nonperiodic_columns)))

In [54]:
column_names

[u'ZZPt',
 u'ZZPhi',
 u'PFMET',
 u'ZZEta',
 u'JetPt',
 u'JetPhi',
 u'D_ZHh_ggH_ME',
 u'Z2Mass',
 u'D_VBF2j_ZHh_ME',
 u'D_VBF2j_WHh_ME',
 u'Z1Mass',
 u'D_WHh_ZHh_ME',
 u'Z1Pt',
 u'ZZMassErr',
 u'JetEta',
 u'D_VBF2j_ggH_ME',
 u'Z2Pt',
 u'D_WHh_ggH_ME']

In [55]:
# synthesize a Pandas frame with the inputs this network expects

In [56]:
variable_ranges = {
    "JetPt": (0.0, 500.0),
    "JetEta": (-3.0, 3.0),
    "JetPhi": (-np.pi, np.pi),
    "ZZPt": (0.0, 200.0),
    "ZZPhi": (-np.pi, np.pi),
    "PFMET": (0.0, 200.0),
    "ZZEta": (-3.0, 3.0),
    "Z2Mass": (12, 60),
    "Z1Mass": (40, 110),
    "ZZMassErr": (0, 6),
    "Z1Pt": (0, 300),
    "Z2Pt": (0, 200),
    "D_ZHh_ggH_ME": (0.0, 1.0),
    "D_VBF2j_ZHh_ME": (0.0, 1.0),
    "D_VBF2j_WHh_ME": (0.0, 1.0),
    "D_WHh_ZHh_ME": (0.0, 1.0),
    "D_VBF2j_ggH_ME": (0.0, 1.0),
    "D_WHh_ggH_ME": (0.0, 1.0)
}

In [57]:
list_inputs = ["JetPt", "JetPhi", "JetEta"]

In [58]:
def generate_random_input(input_columns, fixed_input):
    generated_input = {}
    
    for column_name in input_columns:
        var_low = variable_ranges[column_name][0]
        var_high = variable_ranges[column_name][1]
        
        if column_name in list_inputs:
            generated_input[column_name] = [np.random.uniform(var_low, var_high, 4)]
        else:
            generated_input[column_name] = [np.random.uniform(var_low, var_high)]
    
    generated_input.update(fixed_input)
    
    return generated_input

In [118]:
def evaluate_model(model, pre, input_dict):
    df = pd.DataFrame.from_dict(input_dict)
    pre_data = pre.process(df)
        
    output = model.get_keras_model().predict(x = pre_data, verbose = 2, batch_size = len(pre_data))
    return np.asscalar(output)

In [121]:
fixed_input = {"nCleanedJetsPt30": 2}

In [122]:
start_input = generate_random_input(column_names, fixed_input)

In [123]:
evaluate_model(model, pre, start_input)

0.9999943971633911

In [146]:
total_inputs = sum(map(lambda x: 1 if type(x) is int else max(np.shape(x)), start_input.values()))

In [147]:
def fill_inputs(template, input_list):
    for key, val in template.iteritems():
        

In [149]:
fill_inputs(start_input, None)

D_WHh_ggH_ME
D_VBF2j_WHh_ME
ZZPhi
PFMET
JetPt
JetPhi
D_ZHh_ggH_ME
Z2Mass
D_VBF2j_ZHh_ME
ZZPt
Z1Mass
D_WHh_ZHh_ME
Z1Pt
ZZMassErr
ZZEta
D_VBF2j_ggH_ME
Z2Pt
JetEta
nCleanedJetsPt30
